In [5]:
import numpy as np

# Define Rosenbrock's function
def rosenbrock(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# Define Powell's quartic function
def powell_quartic(x):
    return sum((x[i]**4.0 - 10*x[i]**2.0 + 1)**2 for i in range(len(x)))

# Gradient function to compute finite difference
def gradient(f, x, h=1e-5):
    n = len(x)
    grad = np.zeros(n)
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] -= h
        x2[i] += h
        grad[i] = (f(x2) - f(x1)) / (2 * h)
    return grad

# Fletcher-Reeves step-by-step method
def fletcher_reeves_step_by_step(objective_func, x0, max_iter=1000, tol=1e-6):
    # Line search to find optimal step length
    def line_search(f, x, d):
        alpha = 1.0
        rho = 0.5
        c = 1e-4
        while f(x + alpha * d) > f(x) + c * alpha * np.dot(gradient(f, x), d):
            alpha *= rho
        return alpha

    x = x0
    g = gradient(objective_func, x)
    d = -g

    while np.linalg.norm(g) > tol:
        alpha = line_search(objective_func, x, d)
        x = x + alpha * d

        g_new = gradient(objective_func, x)
        beta = np.dot(g_new, g_new) / np.dot(g, g)
        d = -g_new + beta * d
        g = g_new

    return np.round(x, 4)

# Example usage with Rosenbrock and Powell's Quartic functions
x0_rosenbrock = np.array([1.5, 1.5])
x0_powell = np.array([1.5, 1.5])

rosenbrock_result = fletcher_reeves_step_by_step(rosenbrock, x0_rosenbrock)
powell_result = fletcher_reeves_step_by_step(powell_quartic, x0_powell)

print("Rosenbrock Result:", rosenbrock_result)
print("Powell Quartic Result:", powell_result)


Rosenbrock Result: [1. 1.]
Powell Quartic Result: [0.3178 0.3178]


In [8]:



def marquardt_method(f, grad_f, x0, tol=1e-6, max_iter=100, lambda_init=1e-3):
    x = np.array(x0, dtype=float) 
    lambda_ = lambda_init 

    for iteration in range(max_iter):
        grad = grad_f(f, x)  
        H = np.eye(len(x)) + lambda_ * np.eye(len(x))  

        try:
            delta_x = np.linalg.solve(H, -grad) 
        except np.linalg.LinAlgError:
            raise ValueError("Singular matrix encountered in solving linear system.")

        x_new = x + delta_x
        cost = f(x)
        cost_new = f(x_new)

        if np.linalg.norm(delta_x) < tol:
            return x_new, {"status": "converged", "iterations": iteration + 1, "cost": cost_new}

        if cost_new < cost:
            x = x_new
            lambda_ /= 10
        else:
            lambda_ *= 10

    return x, {"status": "max_iter_exceeded", "iterations": max_iter, "cost": f(x)}

x0_powell = [1.5, 1.5]

solution_powell, info_powell = marquardt_method(
    powell_quartic,
    gradient,
    x0_powell
)

# Print results
print("Powell's Quartic Solution:", solution_powell)
print("Powell's Quartic Info:", info_powell)


Powell's Quartic Solution: [-0.31783718 -0.31783718]
Powell's Quartic Info: {'status': 'converged', 'iterations': 26, 'cost': 3.7821679767702097e-13}


In [ ]:
import numpy as np


def quasi_newton_method(f, grad_f, x0, tol=1e-6, max_iter=100):
    
    x = np.array(x0, dtype=float)
    n = len(x)
    H = np.eye(n)

    for iteration in range(max_iter):
        grad = grad_f(f, x)
        if np.linalg.norm(grad) < tol:
            return x, {"status": "converged", "iterations": iteration, "grad_norm": np.linalg.norm(grad)}

        p = -H @ grad
        alpha = 1.0
        while f(x + alpha * p) > f(x) + 1e-4 * alpha * grad.dot(p):
            alpha *= 0.5

        x_new = x + alpha * p
        grad_new = grad_f(f, x_new)
        s = x_new - x
        y = grad_new - grad

        rho = 1.0 / (y.dot(s))
        if np.isfinite(rho):
            I = np.eye(n)
            H = (I - rho * np.outer(s, y)) @ H @ (I - rho * np.outer(y, s)) + rho * np.outer(s, s)

        x = x_new

    return x, {"status": "max_iter_exceeded", "iterations": max_iter, "grad_norm": np.linalg.norm(grad)}

if __name__ == "__main__":

    x0_rosenbrock = [-1.2, 1.0]
    solution_rosenbrock, info_rosenbrock = quasi_newton_method(rosenbrock, gradient, x0_rosenbrock)
    print("Rosenbrock Solution:", solution_rosenbrock)
    print("Rosenbrock Info:", info_rosenbrock)

    x0_powell = [3.0, -1.0, 0.0, 1.0]
    solution_powell, info_powell = quasi_newton_method(powell_quartic, gradient, x0_powell)
    print("Powell's Quartic Solution:", solution_powell)
    print("Powell's Quartic Info:", info_powell)


Rosenbrock Solution: [0.99999998 0.99999996]
Rosenbrock Info: {'status': 'converged', 'iterations': 34, 'grad_norm': 3.482770683606661e-09}
Powell's Quartic Solution: [ 3.14626437  0.31783724  0.         -0.31783725]
Powell's Quartic Info: {'status': 'converged', 'iterations': 15, 'grad_norm': 8.920314502476037e-07}
